## Prerequisites

In [1]:
from tqdm.auto import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import json
import datasets
import os
import glob

pd.set_option("display.max_colwidth", None)

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument
from transformers import AutoTokenizer
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from ragatouille import RAGPretrainedModel
from langchain_core.vectorstores import VectorStore
from langchain_core.language_models.llms import LLM
from langchain_community.llms import HuggingFaceHub
from langchain_community.document_loaders import PyPDFLoader

In [3]:
# Import the load_dotenv function from the dotenv module
from dotenv import load_dotenv

# Call the load_dotenv function to load environment variables from a .env file
load_dotenv()

os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv("HUGGINGFACEHUB_API_TOKEN")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [4]:
from trulens_eval import Tru
from trulens_eval.tru_custom_app import instrument

/Users/priyanshutuli/Desktop/RAG_pipeline_testing/.venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1007)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1007)>


In [5]:
class RAG_pipeline:

    def __init__(self, data_dir_path: str, chunk_size: int):
        self.data_dir_path = data_dir_path
        self.load_documents(self.data_dir_path)
        self.chunk_size = chunk_size
        self.RAG_PROMPT_TEMPLATE = """
            <|system|>
            Using the information contained in the context,
            give a comprehensive answer to the question.
            Respond only to the question asked, response should be concise and relevant to the question.
            Provide the number of the source document when relevant.
            If the answer cannot be deduced from the context, do not give an answer.</s>
            <|user|>
            Context:
            {context}
            ---
            Now here is the question you need to answer.

            Question: {question}
            </s>
            <|assistant|>
        """
        self.markdown_separators = [
        "\n#{1,6} ",
            "```\n",
            "\n\\*\\*\\*+\n",
            "\n---+\n",
            "\n___+\n",
            "\n\n",
            "\n",
            " ",
            "",
        ]
    
    def load_documents(self, data_dir_path: str):
        docs = []
        for file_path in glob.glob(data_dir_path + "/*.pdf"):
            loader = PyPDFLoader(file_path)
            pages = loader.load_and_split()
            docs.extend(pages)

        self.knowledge_base = [
            LangchainDocument(page_content=page.page_content, metadata=page.metadata) for page in tqdm(docs)]

    def split_documents(self, tokenizer_name: str) -> List[LangchainDocument]:
        """
        Split documents into chunks of size `chunk_size` characters and return a list of documents.
        """
        text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
            AutoTokenizer.from_pretrained(tokenizer_name),
            chunk_size=self.chunk_size,
            chunk_overlap=int(self.chunk_size / 10),
            add_start_index=True,
            strip_whitespace=True,
            separators=self.markdown_separators,
        )

        docs_processed = []
        for doc in self.knowledge_base:
            docs_processed += text_splitter.split_documents([doc])

        # Remove duplicates
        unique_texts = {}
        docs_processed_unique = []
        for doc in docs_processed:
            if doc.page_content not in unique_texts:
                unique_texts[doc.page_content] = True
                docs_processed_unique.append(doc)
    
        return docs_processed_unique
    
    def load_embeddings(self,
        embedding_model_name: Optional[str] = "thenlper/gte-small") -> FAISS:
        """
        Creates a FAISS index from the given embedding model and documents. Loads the index directly if it already exists.

        Args:
            langchain_docs: list of documents
            chunk_size: size of the chunks to split the documents into
            embedding_model_name: name of the embedding model to use

        Returns:
            FAISS index
        """
        # load embedding_model
        embedding_model = HuggingFaceEmbeddings(
            model_name=embedding_model_name,
            multi_process=True,
            model_kwargs={"device": "cpu"},
            encode_kwargs={"normalize_embeddings": True},  # set True to compute cosine similarity
        )

        # Check if embeddings already exist on disk
        index_name = f"index_chunk:{self.chunk_size}_embeddings:{embedding_model_name.replace('/', '~')}"
        index_folder_path = f"./data/indexes/{index_name}/"
        if os.path.isdir(index_folder_path):
            return FAISS.load_local(
                index_folder_path,
                embedding_model,
                distance_strategy=DistanceStrategy.COSINE,
                allow_dangerous_deserialization=True
            )

        else:
            print("Index not found, generating it...")
            docs_processed = self.split_documents(
                embedding_model_name,
            )
            knowledge_index = FAISS.from_documents(
                docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
            )
            knowledge_index.save_local(index_folder_path)
            return knowledge_index
        
    def answer_with_rag(self, question: str,
        llm: LLM,
        knowledge_index: VectorStore,
        reranker: Optional[RAGPretrainedModel] = None,
        num_retrieved_docs: int = 30,
        num_docs_final: int = 7) -> Tuple[str, List[LangchainDocument]]:
        """Answer a question using RAG with the given knowledge index."""
        # Gather documents with retriever
        relevant_docs = knowledge_index.similarity_search(query=question, k=num_retrieved_docs)
        relevant_docs = [doc.page_content for doc in relevant_docs]  # keep only the text

        # Optionally rerank results
        if reranker:
            relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)
            relevant_docs = [doc["content"] for doc in relevant_docs]

        relevant_docs = relevant_docs[:num_docs_final]

        # Build the final prompt
        context = "\nExtracted documents:\n"
        context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)])

        final_prompt = self.RAG_PROMPT_TEMPLATE.format(question=question, context=context)

        # Redact an answer
        answer = llm.invoke(final_prompt)

        return answer, relevant_docs

In [6]:
rag_pipeline = RAG_pipeline(data_dir_path="./data", chunk_size=512)

  0%|          | 0/662 [00:00<?, ?it/s]

In [14]:
knowledge_vector_database = rag_pipeline.load_embeddings()

In [7]:
from langchain_community.llms import HuggingFaceHub

repo_id = "HuggingFaceH4/zephyr-7b-beta"
READER_MODEL_NAME = "zephyr-7b-beta"

READER_LLM = HuggingFaceHub(
    repo_id=repo_id,
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

/Users/priyanshutuli/Desktop/RAG_pipeline_testing/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [8]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import accelerate

model = "meta-llama/Llama-2-7b-chat-hf"

# model = AutoModelForCausalLM.from_pretrained(model)
tokenizer=AutoTokenizer.from_pretrained(model)
llama_pipeline=transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="cpu",
    max_new_tokens=500,
    do_sample=True,
    # num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
    )


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
llama=HuggingFacePipeline(pipeline=llama_pipeline, model_kwargs={"max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03})

In [10]:
# repo_id = "mistralai/Mistral-7B-Instruct-v0.1"
# READER_MODEL_NAME = "Mistral-7B-Instruct-v0.1"

# LLM_CHAT = HuggingFaceHub(
#     repo_id=repo_id,
#     task="text-generation",
#     model_kwargs={
#         "max_new_tokens": 512,
#         "top_k": 30,
#         "temperature": 0.1,
#         "repetition_penalty": 1.03,
#     },
# )

In [11]:
question = "What is CVA estimated based on?"

In [12]:
RERANKER = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

[Apr 18, 00:24:27] Loading segmented_maxsim_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/Users/priyanshutuli/Desktop/RAG_pipeline_testing/.venv/lib/python3.10/site-packages/torch/cuda/amp/grad_scaler.py:126: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


In [15]:
answer, relevant_docs = rag_pipeline.answer_with_rag(question = question, llm = READER_LLM, knowledge_index=knowledge_vector_database, num_docs_final=1,
                                                     reranker=RERANKER)

/Users/priyanshutuli/Desktop/RAG_pipeline_testing/.venv/lib/python3.10/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
100%|██████████| 1/1 [00:05<00:00,  5.19s/it]


In [16]:
answer

'\n            <|system|>\n            Using the information contained in the context,\n            give a comprehensive answer to the question.\n            Respond only to the question asked, response should be concise and relevant to the question.\n            Provide the number of the source document when relevant.\n            If the answer cannot be deduced from the context, do not give an answer.</s>\n            <|user|>\n            Context:\n            \nExtracted documents:\nDocument 0:::\n$393\xa0million , or 40\xa0basis points, in the same period a year \nago , driven by higher losses in all consumer portfolios, \nprimarily in our credit card portfolio.\n• Nonperforming assets (NPAs) of $8.2\xa0billion  at \nSeptember\xa030, 2023 , increased $2.4 billion , or 42% , from \nWells Fargo & Company 5\n            ---\n            Now here is the question you need to answer.\n\n            Question: What is CVA estimated based on?\n            </s>\n            <|assistant|>\n 

In [19]:
answer.split("Answer:")[-1]

' Credit valuation adjustment (CVA) is a risk management concept used to adjust the value of financial instruments to reflect the potential credit losses due to counterparty default. CVA is calculated by estimating the present value of future expected credit losses, discounted at the risk-free rate, and subtracting any collateral received from the counterparty. The calculation takes into account various factors such as the probability of default, recovery rate, and time value of money. CVA is a complex and subjective calculation that involves significant judgment and uncertainty, particularly in estimating the probability of default and recovery rate.'

In [20]:
relevant_docs

['$393\xa0million , or 40\xa0basis points, in the same period a year \nago , driven by higher losses in all consumer portfolios, \nprimarily in our credit card portfolio.\n• Nonperforming assets (NPAs) of $8.2\xa0billion  at \nSeptember\xa030, 2023 , increased $2.4 billion , or 42% , from \nWells Fargo & Company 5']

In [30]:
from llama_cpp import Llama

In [31]:
llm = Llama.from_pretrained(
    repo_id="Qwen/Qwen1.5-0.5B-Chat-GGUF",
    filename="*q8_0.gguf",
    verbose=True,
    local_dir="./models"
)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from ./models/qwen1_5-0_5b-chat-q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.name str              = Qwen1.5-0.5B-Chat-AWQ-fp16
llama_model_loader: - kv   2:                          qwen2.block_count u32              = 24
llama_model_loader: - kv   3:                       qwen2.context_length u32              = 32768
llama_model_loader: - kv   4:                     qwen2.embedding_length u32              = 1024
llama_model_loader: - kv   5:                  qwen2.feed_forward_length u32              = 2816
llama_model_loader: - kv   6:                 qwen2.attention.head_count u32              = 16
llama_model_loader: - kv   7:              qwen2.attentio

In [32]:
from langchain_community.llms import LlamaCpp

llama = LlamaCpp(model_path = "/Users/priyanshutuli/Desktop/RAG_pipeline_testing/models/qwen1_5-0_5b-chat-q8_0.gguf")

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /Users/priyanshutuli/Desktop/RAG_pipeline_testing/models/qwen1_5-0_5b-chat-q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.name str              = Qwen1.5-0.5B-Chat-AWQ-fp16
llama_model_loader: - kv   2:                          qwen2.block_count u32              = 24
llama_model_loader: - kv   3:                       qwen2.context_length u32              = 32768
llama_model_loader: - kv   4:                     qwen2.embedding_length u32              = 1024
llama_model_loader: - kv   5:                  qwen2.feed_forward_length u32              = 2816
llama_model_loader: - kv   6:                 qwen2.attention.head_count u32              = 16
llama_mod